# Program to rate characters
Create a model based on a persons appeal to different characters in Genshin Impact
Following this form: https://forms.office.com/Pages/ResponseDetailPage.aspx?id=tTAKZi6OaUe560ryi_0SvUHGZGPvcItElIw3gzwgl7NUMFJZUE5aNDVORVVMRENTSzFQV0laTjc4Vy4u&rid=4&GetResponseToken=q7rahzF2E7Igy2bHzBfrQBctplWW1STeVmCDaCKbsME&origin=rc

#### inputs

In [ ]:
import torch as pt

